## ESG Research 
### Yfinance, Pandas, NumPy


# Installation
* pip install pandas
* pip install numpy
* pip3 install tqdm
* pip3 install seaborn
* pip install yfinance | pip3 install yfinance
* pip install git+https://github.com/rodrigobercini/yfinance.git


In [1]:
#Import necessary libraries
import pandas as pd
import numpy as np
import yfinance as yf
from pandas import Series, DataFrame
from pandas.tseries import offsets
import csv
import datetime
import csv
from csv import writer
from tqdm import tqdm
import os
import time
from yahoofinancials import YahooFinancials
from sys import platform

dt = datetime.datetime.today()

YEAR = dt.year
MONTH = dt.month
CURRENT_CSV_FILE = bool

if platform == "linux" or platform == "linux2":
       op_sys = "linux"
elif platform == "darwin":
       op_sys = "Mac"
elif platform == "win32":
       op_sys = "Windows"
    
ESG_COLUMNS = {'palmOil', 'controversialWeapons', 'gambling', 'socialScore', 'nuclear',
       'furLeather', 'alcoholic', 'gmo', 'catholic', 'socialPercentile',
       'peerCount', 'governanceScore', 'environmentPercentile',
       'animalTesting', 'tobacco', 'totalEsg', 'highestControversy',
       'esgPerformance', 'coal', 'pesticides', 'adult', 'percentile',
       'peerGroup', 'smallArms', 'environmentScore', 'governancePercentile',
       'militaryContract', 
       #'Stock'
}

ALL_COLUMNS = {'palmOil', 'peerCount', 'environmentScore', 'militaryContract',
       'esgPerformance', 'coal', 'peerGroup', 'furLeather', 'gambling',
       'animalTesting', 'catholic', 'nuclear', 'totalEsg', 'adult',
       'environmentPercentile', 'highestControversy', 'socialScore',
       'percentile', 'alcoholic', 'socialPercentile', 'pesticides',
       'governancePercentile', 'controversialWeapons', 'gmo', 'smallArms',
       'tobacco', 'governanceScore', 'Stock Ticker', 'Sector', 'Name'
}


USER_PATH = '/Users/MichaelCalmette/Documents/Finance Research/ESG Data Files/'
DATE_PATH = "esg-{}-{}.csv".format(dt.month, dt.year)
#DATE_PATH = "esg-2-2022.csv"

FILE_PATH = os.path.join(USER_PATH,DATE_PATH)

Create functions to see if file exists / if there is data in there

In [2]:
f = open(FILE_PATH, 'w')
f.close
### Check if DF is empty

<function TextIOWrapper.close()>

In [2]:
def send_notification(title, t):
    if op_sys == "Mac":
        message = ("Time taken: {} minutes".format(t))
        command = f'''
        osascript -e 'display notification "{message}" with title "{title}"'
        '''
        os.system(command)
    

Need to compare data in esg-csv to all tickers

In [3]:
df = pd.read_csv('tickers.csv')
#all_data = pd.DataFrame(columns=ESG_COLUMNS)
# Only do if month is partially filled
all_data = pd.read_csv(FILE_PATH)
empty_stocks = pd.DataFrame(columns= ['Ticker'])

In [4]:
tic = df.sample(n=1)
ticker_sym = tic.loc[tic.index[0],'Symbol'] # get ticker name ex: 'AAPL'
print(ticker_sym)
ticker = yf.Ticker(ticker_sym).history(period="1mo")
ticker

SJM


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-03-14,129.220001,130.910004,128.009995,128.929993,877000,0,0
2022-03-15,129.110001,129.490005,127.230003,129.300003,994600,0,0
2022-03-16,128.880005,129.360001,127.199997,129.339996,776200,0,0
2022-03-17,129.809998,130.600006,128.339996,129.580002,551900,0,0
2022-03-18,129.720001,129.720001,127.010002,128.070007,1423800,0,0
2022-03-21,128.630005,130.529999,128.630005,130.169998,792900,0,0
2022-03-22,130.529999,130.889999,128.750000,130.300003,737200,0,0
2022-03-23,131.190002,132.770004,130.160004,130.179993,826100,0,0
2022-03-24,130.220001,131.940002,130.220001,131.389999,503000,0,0


Yahoo Financials to get additional stock data

In [ ]:
yahoo_financials = YahooFinancials('AAPL')
print(yahoo_financials.get_summary_data())

Test pulling a random ticker and getting sustainability information

In [ ]:
tic = df.sample(n=1)
ticker_sym = tic.loc[tic.index[0],'Symbol'] # get ticker name ex: 'AAPL'
ticker = yf.Ticker(ticker_sym)
y = ticker.sustainability 
y

In [13]:
df1_transposed = pd.DataFrame()
no_data_stocks = 0
start_time = time.time()
temp = all_data[['Stock Ticker','Sector','Name']].copy()

for x in tqdm(range(100)):
    df_test = df.merge(temp, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']
    tic = df_test.sample(n=1)
    ticker_sym = tic.loc[tic.index[0],'Symbol'] # get ticker name ex: 'AAPL'
    ticker = yf.Ticker(ticker_sym)
    y = ticker.sustainability 
    df1 = y

    if df1 is None:
        no_data_stocks += 1
        empty_stocks = empty_stocks.append({"Ticker": ticker_sym}, ignore_index = True)
    else:
        df1_transposed = df1.transpose()
        #print(df1_transposed.columns)
        df1_transposed['Stock Ticker'] = ticker_sym
        df1_transposed['Sector'] = tic.loc[tic.index[0],'Sector']
        df1_transposed['Name'] = tic.loc[tic.index[0],'Name']

        #all_data = df1_transposed
        all_data = all_data.append(df1_transposed, ignore_index = True)
    #temp = temp[temp['Stock Ticker'] != ticker_sym]

all_shape = len(all_data)
empty_shape = len(empty_stocks)
time_taken = round((time.time() - start_time) / 60,2)

print("There are: {} stocks left in the database".format(len(df_test)))
print("There are: {} stocks with no ESG data".format(no_data_stocks))

print("Dataframe shape: {}".format(all_shape))
print("Missing stocks: {}".format(empty_shape))

send_notification("Completed ESG Data Pull",time_taken)

100%|██████████| 100/100 [18:36<00:00, 11.17s/it]


There are: 66 stocks left in the database
There are: 95 stocks with no ESG data
Dataframe shape: 543
Missing stocks: 357


Make sure there are no duplicates

In [ ]:
all_data

In [14]:
s = len(all_data)
print(all_data.shape)
all_data = all_data.drop_duplicates()
dropped=len(all_data) - s
print("{} duplicates have been dropped".format(dropped))

(543, 30)
-100 duplicates have been dropped


In [15]:
all_data.shape

(443, 30)

In [16]:
all_data['Stock Ticker'].value_counts()

BR      1
NEM     1
MPC     1
SBAC    1
HAS     1
       ..
NFLX    1
STT     1
CCL     1
EXPE    1
KMI     1
Name: Stock Ticker, Length: 443, dtype: int64

In [17]:
if os.stat(FILE_PATH).st_size == 0:
    CURRENT_CSV_FILE = 0
    print('ESG File is empty')
    f = open(FILE_PATH, 'w')
    f.close

    with open(FILE_PATH, 'a', newline='') as f_object: 
        writer_object = writer(f_object) # Pass the CSV  file object to the writer() function
        writer_object.writerow(all_data) # Pass the data in the list as an argument into the writerow() function
        f_object.close()
    print('Column names have been entered.')

else:
    CURRENT_CSV_FILE = 1
    print('ESG File has column headers.')
    all_data.to_csv(FILE_PATH, mode='a', index=False, header=False)
    print('ESG Data has been entered.')

ESG File has column headers.
ESG Data has been entered.
